In [26]:
import sqlite3
import pickle
import pandas as pd

In [27]:
shutuba_table = pd.read_pickle("doyou_score")
# shutuba_table.drop(columns=['斤量','jockey_id', 'trainer_id', '体重'\
#                             , '体重変化', 'n_horses', 'course_len', 'weather' \
#                             , 'race_type', 'ground_state', 'around'], inplace=True)

In [28]:
print(shutuba_table.columns)

Index(['枠番', '馬番', '単勝', 'horse_id', '馬名', '騎手名', '性', '体重変化', '年齢', 'date',
       'race_class', 'race_name', '開催', 'start_time', 'oddsrank_score',
       'speed_score', 'rank1to1_score'],
      dtype='object')


In [29]:
shutuba_table = shutuba_table.astype(str) # 全体を文字列に
shutuba_table.reset_index(inplace=True)
shutuba_table = shutuba_table.rename(columns={'index': 'race_id'}) # インデックスを列に

In [30]:
conn = sqlite3.connect('races.sqlite')
cursor = conn.cursor()

**********

runnersテーブルのodds

In [31]:
runner_table = shutuba_table[['race_id', '馬番', '枠番', '騎手名', '単勝', 'horse_id']].copy()
# runner_table = shutuba_table[['race_id', '馬番', '枠番', '騎手名', '単勝', 'horse_id']].copy()
runner_table.rename(columns={'馬番' : 'number', 
                             '枠番' : 'position',
                             '騎手名' : 'jockey',
                             '単勝' : 'odds'}, 
                  inplace=True)

In [ ]:
# cursor.execute('SELECT * FROM runners')
# data = cursor.fetchall()

# # データを表示
# for row in data:
#     print(row)

In [33]:
ids = shutuba_table['race_id'].unique()
for i in range(ids.shape[0]):
    conn.execute("DELETE from runners where race_id=?;", (ids[i],))
conn.commit()

In [34]:
# 新しいオッズにするために、該当のrace_idのデータを一度すべて消して新しく書き込む
for i in range(ids.shape[0]):
    new_odds_table = runner_table[runner_table['race_id'] == ids[i]]
    new_odds_table.to_sql('runners', conn, if_exists='append', index=None)


*****************

predictsテーブル

In [35]:
pred_table = shutuba_table[['race_id', '馬番', 'oddsrank_score', 'rank1to1_score', 'speed_score', '開催']].copy()

In [36]:
pred_table.rename(columns={'馬番' : 'runners_number', 
                           'oddsrank_score' : 'model1',
                            'rank1to1_score' : 'model2',
                            'speed_score' : 'model3',
                        #   'モデル4' : 'model4'
                            '開催' : 'model4',}, 
                  inplace=True)

In [39]:
pred_table.to_sql('predicts', conn, if_exists='append', index=None)

479

*************

In [40]:
conn.close()